In [1]:
import openrouteservice as ors
import folium
import folium.plugins as plugins
import random

client = ors.Client(key='5b3ce3597851110001cf6248facc383c642b420fa9818331b313169e')

In [2]:
import pandas as pd
# Read Data
nodes = pd.read_csv("SolutionData\\Nodes.csv")
truckRoute = pd.read_csv("SolutionData\\TruckRoute.csv")
droneRoute = pd.read_csv("SolutionData\\DroneRoute.csv")
nodes

,Index,X,Y,Type
0,0,29.442315,40.805331,TRUCK
1,1,29.245976,40.969152,UAVELIGIBLE
2,2,29.250962,40.963022,UAVELIGIBLE
3,3,29.278775,40.977839,UAVELIGIBLE
4,4,28.851020,41.025410,TRUCK
5,5,28.852149,41.108290,UAVELIGIBLE
6,6,29.160179,41.171734,TRUCK
7,7,29.442315,40.805331,TRUCK


In [3]:
m = folium.Map(location=[40.80533085, 29.442315], tiles='cartodbpositron', zoom_start=13)

# Add Nodes
droneEligableChance = 0.7
for index, row in nodes.iterrows():
    color = None
    if(row["Type"] == "UAVELIGIBLE"):
        color = "orange"
    else:
        color = "cyan"
    if(index == 0 or index == nodes.shape[0]-1):
        icon = plugins.BeautifyIcon(icon = "truck", iconShape="marker", backgroundColor="white")
    else:
        icon = plugins.BeautifyIcon(icon = "", iconShape="marker", number = row["Index"], backgroundColor=color)
    folium.Marker([row["Y"], row["X"]], icon=icon, popup= row["Index"],).add_to(m)
m

In [4]:
truckRoute

,Index,X,Y,CompletionTime
0,0,29.442315,40.805331,60.000000
1,4,28.851020,41.025410,3308.385000
2,6,29.160179,41.171734,5806.275000
3,7,29.442315,40.805331,8660.724853


In [5]:
# Add Truck Routes

for i in range(0, truckRoute.shape[0] - 1):
    popup = "{i} --> {j}"
    
    coordinates = [[truckRoute["X"][i], truckRoute["Y"][i]], [truckRoute["X"][i+1], truckRoute["Y"][i+1]]]
    route = client.directions(
    coordinates=coordinates,
    profile='driving-car',
    format='geojson',
    validate=False,)
    plugins.AntPath(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']], 
                           tooltip = popup.format(i = truckRoute["Index"][i], j = truckRoute["Index"][i+1]), linecap ="round").add_to(m)
   
m

In [6]:
droneRoute

,DroneIndex,iIndex,jIndex,kIndex,iX,iY,jX,jY,kX,kY,Launch[i],Arrival[j],Launch[j],Retrieval[k]
0,0,0,1,4,29.442315,40.805331,29.245976,40.969152,28.851020,41.025410,60.0000,731.4350,791.435000,3160.590
1,0,4,3,6,28.851020,41.025410,29.278775,40.977839,29.160179,41.171734,3220.5900,4520.7850,4580.785000,5716.275
2,0,6,5,7,29.160179,41.171734,28.852149,41.108290,29.442315,40.805331,5806.2750,6735.7675,6795.767434,8660.725
3,1,4,2,7,28.851020,41.025410,29.250962,40.963022,29.442315,40.805331,3309.4875,4521.3400,4581.339934,8630.725


In [7]:
# Add Drone Routes
colorsForDroneRoutes = ["red", "purple", "yellow", "green", "orange"]

for rowNumber in range(0, droneRoute.shape[0]):
    i = (droneRoute["iY"][rowNumber], droneRoute["iX"][rowNumber])
    j = (droneRoute["jY"][rowNumber], droneRoute["jX"][rowNumber])
    k = (droneRoute["kY"][rowNumber], droneRoute["kX"][rowNumber])
    
    color = colorsForDroneRoutes[droneRoute["DroneIndex"][rowNumber]]
    
    ijRouteToolTip = "{i} --> {j} | LaunchTime[{i}]: {launch} / ArrivalTime[{j}]: {arrival}"
    jkRouteToolTip = "{j} --> {k} | LaunchTime[{j}]: {launch} / RetrievalTime[{k}]: {retrieval}"
    plugins.AntPath(locations = [i,j], color=color, tooltip = ijRouteToolTip.format(i = droneRoute["iIndex"][rowNumber],
                                                                                   j = droneRoute["jIndex"][rowNumber],
                                                                                   launch = droneRoute["Launch[i]"][rowNumber],
                                                                                   arrival = droneRoute["Arrival[j]"][rowNumber])).add_to(m)
    plugins.AntPath(locations = [j,k], color=color, tooltip = jkRouteToolTip.format(j = droneRoute["jIndex"][rowNumber],
                                                                                   k = droneRoute["kIndex"][rowNumber],
                                                                                   launch = droneRoute["Launch[j]"][rowNumber],
                                                                                   retrieval = droneRoute["Retrieval[k]"][rowNumber])).add_to(m)
m    

In [8]:
m.save('map.html')

In [9]:
route["features"][0]["properties"]["summary"]

{'distance': 65578.7, 'duration': 2794.4}